In [1]:
import pandas as pd
import statsmodels.api as sm
import linearmodels as lm
import pyreadstat
from statsmodels.stats.diagnostic import het_breuschpagan, het_white
from scipy.stats import pearsonr
from linearmodels.panel import compare


In [2]:
import sys
sys.path.append('/Applications/Stata/utilities')
from pystata import config
config.init('se')


  ___  ____  ____  ____  ____ ©
 /__    /   ____/   /   ____/      17.0
___/   /   /___/   /   /___/       SE—Standard Edition

 Statistics and Data Science       Copyright 1985-2021 StataCorp LLC
                                   StataCorp
                                   4905 Lakeway Drive
                                   College Station, Texas 77845 USA
                                   800-STATA-PC        https://www.stata.com
                                   979-696-4600        stata@stata.com

Stata license: Unlimited-user network, expiring 21 Sep 2023
Serial number: 401709301720
  Licensed to: Dhruv Akshay Pandit
               ISEG

Notes:
      1. Unicode is supported; see help unicode_advice.
      2. Maximum number of variables is set to 5,000; see help set_maxvar.


In [3]:
data_stata, meta = pyreadstat.read_dta(r'Microeconometrics/credit_loans/T2.dta')
label_dict = {}
for var_name, label in zip(meta.column_names, meta.column_labels):
    label_dict[var_name] = label


In [4]:
data_stata

,id,year,int_rate_dep,int_rate_loan,disp_inc,loans,td1,td2,td3,td4,td5,td6,log_loans,log_disp_inc,_est_POLS,_est_FE,_est_FD
0,115,2001,0.032813,0.098876,45614.785156,31494.554688,1,0,0,0,0,0,10.357570,10.727987,0,0,0
1,115,2002,0.023960,0.101956,51928.710938,36221.781250,0,1,0,0,0,0,10.497416,10.857627,1,1,0
2,115,2003,0.017812,0.100249,59799.742188,38469.093750,0,0,1,0,0,0,10.557611,10.998756,1,1,1
3,115,2004,0.013986,0.084845,59624.800781,37521.183594,0,0,0,1,0,0,10.532660,10.995827,1,1,1
4,115,2005,0.014953,0.082235,61078.964844,35589.898438,0,0,0,0,1,0,10.479817,11.019923,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,13001,2002,0.020008,0.094448,27942.398438,18070.183594,0,1,0,0,0,0,9.802018,10.237901,1,1,0
1712,13001,2003,0.013324,0.076862,29676.564453,17348.701172,0,0,1,0,0,0,9.761273,10.298113,1,1,1
1713,13001,2004,0.010873,0.076971,29102.466797,17767.171875,0,0,0,1,0,0,9.785108,10.278578,1,1,1
1714,13001,2005,0.012434,0.073306,29503.382812,17358.923828,0,0,0,0,1,0,9.761862,10.292260,1,1,1


In [5]:
obs_counts = data_stata.groupby('id')['year'].count()
if obs_counts.std() > 0: print('Unbalanced Data') 
else: print('Balanced')

Balanced


In [7]:
data_stata['year'] = pd.to_datetime(data_stata['year'], format='%Y')

In [8]:
data_stata.describe()

,id,int_rate_dep,int_rate_loan,disp_inc,loans,td1,td2,td3,td4,td5,td6,log_loans,log_disp_inc,_est_POLS,_est_FE,_est_FD
count,1716.000000,1716.000000,1716.000000,1.716000e+03,1.716000e+03,1716.000000,1716.000000,1716.000000,1716.000000,1716.000000,1716.000000,1716.000000,1716.000000,1716.000000,1716.000000,1716.000000
mean,7219.583916,0.022554,0.078934,5.266329e+05,2.578959e+05,0.166667,0.166667,0.166667,0.166667,0.166667,0.166667,11.411742,12.171405,0.833333,0.833333,0.666667
std,3989.133266,0.008304,0.014130,1.123511e+06,5.103316e+05,0.372787,0.372787,0.372787,0.372787,0.372787,0.372787,1.458747,1.417642,0.372787,0.372787,0.471542
min,115.000000,0.005295,0.048190,5.165132e+03,8.050737e+02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.690934,8.549686,0.000000,0.000000,0.000000
25%,3411.000000,0.016384,0.068641,6.442156e+04,3.310069e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.407307,11.073204,1.000000,1.000000,0.000000
50%,7966.000000,0.021024,0.077324,1.884781e+05,8.408274e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.339557,12.146736,1.000000,1.000000,1.000000
75%,11019.000000,0.027730,0.086999,5.535071e+05,2.536501e+05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.443710,13.224030,1.000000,1.000000,1.000000
max,13001.000000,0.049682,0.144407,1.527107e+07,5.032359e+06,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,15.431399,16.541471,1.000000,1.000000,1.000000


In [9]:
#Creating MultiIndex Dataframe on Id and Year
data_df_multi = data_stata.set_index(['id', 'year'])

In [ ]:
#Creating MultiIndex Dataframe on Id and Year
data_df_multi = data_stata.set_index(['id', 'year'])